In [2]:
import pandas as pd
import numpy as np

In [3]:
## Stemming and Lemmatization

In [4]:
# install the nltk (natural language toolkit) library
!pip install nltk

In [5]:
# training data
# sentences
X_train = ['I love the book', 
           'This is a great book',
          'The fit is great',
          'I love the shoes']
# topics of sentences
y_train = ['books', 'books', 'clothings', 'clothings']

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [7]:
vect = CountVectorizer()
vect.fit(X_train)
# document term matrix. Matrix with the counts
X_train_dtm = vect.transform(X_train)
pd.DataFrame(X_train_dtm.toarray(), 
             columns=vect.get_feature_names_out())

,book,fit,great,is,love,shoes,the,this
0,1,0,0,0,1,0,1,0
1,1,0,1,1,0,0,0,1
2,0,1,1,1,0,0,1,0
3,0,0,0,0,1,1,1,0


In [8]:
X_test = ['I like the book', 
          'Shoes are alright', 
          'I love the books',
         'I lost a shoe']

In [9]:
nb_clf = MultinomialNB()
nb_clf.fit(X=X_train_dtm, y=y_train)

MultinomialNB()

In [10]:
X_test_dtm = vect.transform(X_test)
nb_clf.predict(X_test_dtm)

array(['books', 'clothings', 'clothings', 'books'], dtype='<U9')

## Stemming

In [11]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Derrick\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Derrick\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Derrick\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Derrick\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Derrick\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [12]:
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

In [13]:
# initialize the stemmer
stemmer = PorterStemmer()
print(stemmer.stem('books'))
print(stemmer.stem('reading'))

book
read


In [14]:
phrase = 'I love the books'
words = word_tokenize(phrase)

In [15]:
stemmed_words = [stemmer.stem(word) for word in words] # some nice python syntax
stemmed_words

['i', 'love', 'the', 'book']

In [16]:
' '.join(stemmed_words)

'i love the book'

## Lemmatization

In [17]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

In [18]:
lemmatizer = WordNetLemmatizer()

In [19]:
# needs parts of speech
lemmatizer.lemmatize('eats', pos='v') # v for verb

'eat'

In [20]:
words

['I', 'love', 'the', 'books']

In [21]:
nltk.pos_tag(words) # tells if a word is a verb (VBP) or a noun (NNS)

[('I', 'PRP'), ('love', 'VBP'), ('the', 'DT'), ('books', 'NNS')]

In [22]:
# parts of speech processing function
def process_pos(pos):
    if pos.startswith('J'): # adjective
        return wordnet.ADJ
    elif pos.startswith('V'): # verb
        return wordnet.VERB
    elif pos.startswith('N'): # noun
        return wordnet.NOUN
    elif pos.startswith('R'): #adverb
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [23]:
nltk.pos_tag(words)

[('I', 'PRP'), ('love', 'VBP'), ('the', 'DT'), ('books', 'NNS')]

In [24]:
lemmatized_words = [lemmatizer.lemmatize(word, pos=process_pos(pos))
                   for word,pos in nltk.pos_tag(words)]

In [25]:
phrase = 'example of sentence removal of stopwords'

In [26]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words

words = word_tokenize(phrase)
stripped_phrase = [word for word in words if word not in stop_words]
' '.join(stripped_phrase)

'example sentence removal stopwords'

## Punctuation Removal

In [27]:

import string
punctuation = [punc for punc in string.punctuation]
punctuation

['!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 '{',
 '|',
 '}',
 '~']

In [28]:
phrase = "Hello! how are you?"
words = word_tokenize(phrase)
stripped_phrease = [word for word in words if word not in punctuation]
' '.join(stripped_phrase)

'example sentence removal stopwords'

## Yelp reviews


In [29]:
url = 'https://raw.githubusercontent.com/um-perez-alvaro/Data-Science-Practice/master/Data/yelp.csv'
yelp = pd.read_csv(url)[['text','stars']]
yelp.head()

,text,stars
0,My wife took me here on my birthday for breakf...,5
1,I have no idea why some people give bad review...,5
2,love the gyro plate. Rice is so good and I als...,4
3,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",5
4,General Manager Scott Petello is a good egg!!!...,5


In [30]:
yelp.stars.value_counts()

4    3526
5    3337
3    1461
2     927
1     749
Name: stars, dtype: int64

In [31]:
# Filter to 5- and 1- star reviews
yelp = yelp.loc[yelp.stars.isin([1,5])]
yelp.tail(5)

,text,stars
9990,Yes I do rock the hipster joints. I dig this ...,5
9991,Only 4 stars? \n\n(A few notes: The folks that...,5
9992,I'm not normally one to jump at reviewing a ch...,5
9994,Let's see...what is there NOT to like about Su...,5
9999,4-5 locations.. all 4.5 star average.. I think...,5


In [32]:
text = yelp.loc[yelp.stars==5].iloc[0].text
text

'My wife took me here on my birthday for breakfast and it was excellent.  The weather was perfect which made sitting outside overlooking their grounds an absolute pleasure.  Our waitress was excellent and our food arrived quickly on the semi-busy Saturday morning.  It looked like the place fills up pretty quickly so the earlier you get here the better.\n\nDo yourself a favor and get their Bloody Mary.  It was phenomenal and simply the best I\'ve ever had.  I\'m pretty sure they only use ingredients from their garden and blend them fresh when you order it.  It was amazing.\n\nWhile EVERYTHING on the menu looks excellent, I had the white truffle scrambled eggs vegetable skillet and it was tasty and delicious.  It came with 2 pieces of their griddled bread with was amazing and it absolutely made the meal complete.  It was the best "toast" I\'ve ever had.\n\nAnyway, I can\'t wait to go back!'

In [33]:
# Process the text
text = yelp.loc[0,'text']
words = word_tokenize(text)
words = [word.lower() for word in words]
lemmatized_words = [lemmatizer.lemmatize(word,pos=process_pos(pos))
                   for word,pos in nltk.pos_tag(words)
                   if word not in stop_words and word not in punctuation
                   ]
' '.join(lemmatized_words)

"wife take birthday breakfast excellent weather perfect make sit outside overlook ground absolute pleasure waitress excellent food arrive quickly semi-busy saturday morning look like place fill pretty quickly early get good favor get bloody mary phenomenal simply best 've ever 'm pretty sure use ingredient garden blend fresh order amaze everything menu look excellent white truffle scramble egg vegetable skillet tasty delicious come 2 piece griddle bread amaze absolutely make meal complete best `` toast '' 've ever anyway ca n't wait go back"

In [34]:
yelp['processed_text'] = yelp.text

In [35]:
def process_text(text):
    words = word_tokenize(text)
    words = [word.lower() for word in words]
    lemmatized_words = [lemmatizer.lemmatize(word,pos=process_pos(pos))
                       for word,pos in nltk.pos_tag(words)
                       if word not in stop_words and word not in punctuation
                       ]
    return(' '.join(lemmatized_words))

In [36]:
process_text(text)

"wife take birthday breakfast excellent weather perfect make sit outside overlook ground absolute pleasure waitress excellent food arrive quickly semi-busy saturday morning look like place fill pretty quickly early get good favor get bloody mary phenomenal simply best 've ever 'm pretty sure use ingredient garden blend fresh order amaze everything menu look excellent white truffle scramble egg vegetable skillet tasty delicious come 2 piece griddle bread amaze absolutely make meal complete best `` toast '' 've ever anyway ca n't wait go back"

In [37]:
yelp['processed_text'] = yelp.text.apply(process_text)

KeyboardInterrupt: 

In [ ]:
yelp

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB # or any other classifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split


In [ ]:
X = yelp.processed_text
y = yelp.stars

In [39]:
X.head(5)

NameError: name 'X' is not defined

In [136]:
X_train,X_test,y_train,y_test = train_test_split(X,y)

In [151]:
pipe = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(max_features = 1000, ngram_range=(1,2))), # idk what ngram range is. less features better for TfidVectorizer
    ('clf', MultinomialNB())
])

In [152]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(max_features=1000, ngram_range=(1, 2))),
                ('clf', MultinomialNB())])

In [153]:
y_test_pred = pipe.predict(X_test)
confusion_matrix(y_test,y_test_pred)

array([[ 61, 138],
       [  1, 822]], dtype=int64)

## Grid search on TfidfVectorizer

In [154]:
params_dic = {'vectorizer__max_features':[500,1000,2000,4000],
             'vectorizer__ngram_range': [(1,1), (1,2)],
             'vectorizer__use_idf': [False, True], # False (CountVectorizer), True (TfidfVectorizer)
              'clf__alpha': [0.1,0.25,0.5,0.75],
             }

In [155]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(pipe, params_dic, cv=5, n_jobs=-1, scoring='roc_auc', verbose=2)
grid.fit(X_train, y_train)

Fitting 5 folds for each of 64 candidates, totalling 320 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vectorizer',
                                        TfidfVectorizer(max_features=1000,
                                                        ngram_range=(1, 2))),
                                       ('clf', MultinomialNB())]),
             n_jobs=-1,
             param_grid={'clf__alpha': [0.1, 0.25, 0.5, 0.75],
                         'vectorizer__max_features': [500, 1000, 2000, 4000],
                         'vectorizer__ngram_range': [(1, 1), (1, 2)],
                         'vectorizer__use_idf': [False, True]},
             scoring='roc_auc', verbose=2)

In [157]:
grid.best_params_
best_pipe = grid.best_estimator_
y_test_pred = best_pipe.predict(X_test)
confusion_matrix(y_test,y_test_pred)

array([[104,  95],
       [  4, 819]], dtype=int64)

## How does the Naive Bayes model choose 5 stars and 1 star?

In [160]:
# store the vocabulary
words = best_pipe['vectorizer'].get_feature_names_out()


In [161]:
pipe['clf'].classes_

array([1, 5], dtype=int64)

In [162]:
# number of times each word appears across all 1 star doc
bad_word_count = best_pipe['clf'].feature_count_[0,:]
good_word_count = best_pipe['clf'].feature_count_[1,:]

In [163]:
words_df = pd.DataFrame({'word':words,
                        'bad':bad_word_count,
                         'good':good_word_count}).set_index('word')
words_df

,bad,good
word,,
00,1.747054,3.001299
000,0.340837,0.576056
10,5.759926,14.628687
10 15,0.164967,0.703823
10 min,0.530048,0.271188
...,...,...
yum,0.000000,7.657545
yum yum,0.000000,0.931869
yummy,0.173559,11.716046


In [164]:
# Add 1 to the columns
words_df = words_df + 1

In [165]:
words_df

,bad,good
word,,
00,2.747054,4.001299
000,1.340837,1.576056
10,6.759926,15.628687
10 15,1.164967,1.703823
10 min,1.530048,1.271188
...,...,...
yum,1.000000,8.657545
yum yum,1.000000,1.931869
yummy,1.173559,12.716046


In [166]:
# convert counts into frequencies
words_df.bad = words_df.bad/words_df.bad.sum()
words_df.good = words_df.good/words_df.good.sum()

In [167]:
# ratios
words_df['bad_ratio'] = words_df.bad/words_df.good
words_df['good_ratio'] = words_df.good/words_df.bad

In [168]:
words_df.sort_values(by='good_ratio', ascending=False).head(20)

,bad,good,bad_ratio,good_ratio
word,,,,
awesome,0.000194,0.002199,0.088173,11.341296
great,0.000961,0.010377,0.092566,10.803152
favorite,0.000211,0.002276,0.092864,10.768386
love,0.000717,0.006837,0.104798,9.542171
excellent,0.000223,0.001870,0.119109,8.395654
fantastic,0.000139,0.001159,0.120376,8.307282
perfect,0.000164,0.001342,0.121869,8.205523
amazing,0.000221,0.001734,0.127226,7.860036
delicious,0.000377,0.002866,0.131585,7.599648


In [169]:
# find some text with 'mozzerella'
yelp.loc[yelp.processed_text.str.contains('mozzarella')]

,text,stars,processed_text
30,"Disclaimer: Like many of you, I am a sucker fo...",5,disclaimer like many sucker charm little home ...
138,I'm from Chicago so I'm picky with my pizza--t...,5,'m chicago 'm picky pizza -- place right ny st...
205,"On one of my many visits to see mi amore, he t...",5,one many visit see mi amore take fantastic lit...
452,WOW this place is good! SO good! And not jus...,5,wow place good good yummy good intrinsically g...
816,I love this place! I love that it's to-go only...,5,love place love 's to-go 3 hour wait like pizz...
913,"The new Harkins Cine Capri, one of the first t...",5,new harkins cine capri one first thing open te...
1189,I have been to this place many times and the f...,5,place many time food service always great exci...
1222,"Honestly, this is the best pizza that I've had...",5,honestly best pizza 've arizona 'm sucker wood...
1611,Great product! I was on a mission to make home...,5,great product mission make homemade mozzarella...
1650,"Delicious food, amazing martini's, and wonderf...",5,delicious food amaze martini 's wonderful dess...


## Inspect a misclassification

In [174]:
X_test[(y_test==1)&(y_test_pred==5)].iloc[10]

"really excite event maybe expectation high really underwhelmed little shade run water time get 1pm obligation 20 vendor food time eat way everyone else already 60 buck n't think mention `` person charge '' patronize invalidate keep tell happy chef well 'm consumer big guy sorry devour wish could well food try pretty good"